In [3]:
import os
import cv2
import numpy as np
import torch
from torchvision import transforms, models, datasets
from torchsummary import summary
from sklearn.neighbors import NearestNeighbors
import torch.nn as nn
from torch.nn import modules

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
model = models.inception_v3(pretrained=True).to('cuda')

In [6]:
def rem_dummy(image):
    mask = image[:, :, -1]
    img = image[:, :, :3]
    img = cv2.bitwise_and(img, img, mask=mask)
    return img

In [7]:
from torch.utils.data import Dataset
import cv2
import os
import numpy as np

class MyDataset(Dataset):
    def __init__(self, txtfile, root, transform=None):
        self.names = open(txtfile, 'r').read().splitlines()
        self.root = root
        self.transform = transform
        
    def __len__(self):
        return len(self.names)
    
    def __getitem__(self, index):
        img_path = os.path.join(self.root, self.names[index])
        img = cv2.imread(img_path, -1)
        img = rem_dummy(img)
        img = cv2.resize(img, (224, 224), interpolation = cv2.INTER_AREA)
        if self.transform is not None:
            img = self.transform(img)
            
        return img

In [8]:
transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406), 
                                                         (0.229, 0.224, 0.225))
                               ])
batch_size = 64
txtfile = r"C:\Users\Area 51\Desktop\Project\data\img_names.txt"
rootdir = r"C:\Users\Area 51\Desktop\Project\data\bottoms_resized_png"

dataset = MyDataset(txtfile, rootdir, transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size)

In [9]:
def feature_matrix(model, dataloader):
    model.eval()

    demo_tnsr = torch.randn((1, 3, 224, 224)).to('cuda')
    demo = model(demo_tnsr)
    emb_mat = torch.randn_like(demo).cpu()
    with torch.no_grad():
        for images in dataloader:

            images = images.to(device)

            feat_mat = model(images).cpu()
            
            emb_mat = torch.cat((emb_mat, feat_mat), 0)
    torch.save(emb_mat, r"C:\Users\Area 51\Desktop\embeddings.pt")        
    return emb_mat

In [10]:
def similar_images(image, num_images, emb_mat):
    image = rem_dummy(image)
    image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)
    image = transforms.ToTensor()(image)
    image = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))(image)
    image = image.unsqueeze(0).to(device)
    
    with torch.no_grad():
        feat_mat = model(image).cpu()
    feat_mat = feat_mat.reshape((feat_mat.shape[0], -1))
    emb_mat = emb_mat.reshape((emb_mat.shape[0], -1))
    
    knn = NearestNeighbors(n_neighbors=num_images, metric='cosine')
    knn.fit(emb_mat)
    
    _, ind = knn.kneighbors(feat_mat)
    ind_lst = ind.tolist()[0][1:]

    return ind_lst

In [12]:
emb_mat = feature_matrix(model, dataloader)

c:\programdata\miniconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [13]:
image = cv2.imread(r"C:\Users\Area 51\Desktop\Project\data\bottoms_resized_png\13583498OWD.png", -1)
ind_lst = similar_images(image, 11, emb_mat)

In [14]:
names = open(txtfile, 'r').read().splitlines()
import numpy as np
c = 1
input_img = cv2.imread(r"C:\Users\Area 51\Desktop\Project\data\bottoms_resized_png\13583498OWD.png", -1)
inp = cv2.resize(input_img, None, fx=2, fy=2)
inp = rem_dummy(inp)
p = []
for i in ind_lst:
    img = cv2.imread(os.path.join(rootdir, names[i-1]), -1)
    
    img = rem_dummy(img)
    p.append(img)

a = np.concatenate((p[0], p[1], p[2], p[3], p[4]), axis=1)
b = np.concatenate((p[5], p[6], p[7], p[8], p[9]), axis=1)
c = np.concatenate((a, b), axis=0)
d = np.concatenate((inp, c), axis=1)

In [15]:
cv2.imwrite(r"C:\Users\Area 51\Desktop\result.jpg", d)

True